In [ ]:
import os
import shutil
from typing import List, Dict

# Script for renaming and moving solution files from tester_results to data

In [ ]:
source_path = "/Users/petr/Documents/Projects/Python/AlgorithmTester/analysis/tester_results/DataAnalysis"
dest_path = "/Users/petr/Documents/Projects/Python/AlgorithmTester/data/DataAnalysis"

dry_run: bool = True

for root, dirs, files in os.walk(source_path):
    print(f"{root}")
    dir_name: str = root.split("/")[-1]
    for filename in files:
        if "column" in filename or "DPWeight" not in filename:
            continue
                   
        split = filename.split("_")
        split = [split[0], split[1], split[4]]
        new_name: str = '_'.join(split)
            
        print(f'{dest_path}/{dir_name}/{split[1]}/{new_name}')
            
        if not dry_run:
            os.rename(f'{root}/{filename}', f'{dest_path}/{dir_name}/{split[1]}/{new_name}')

# Script for aggregating SAT solutions from multiple files to one

In [ ]:
def aggregate_files(results_path: str, output_file_path: str):
    with open(output_file_path, "w") as output_file:
        for root, _, files in os.walk(results_path):
            for file in files:
                if "column" in file:
                    continue
                res_file = open(f'{root}/{file}')
                output_file.write(res_file.read())
                res_file.close()

In [ ]:
datasets = [
    "tester_results_V1_lessAccurate",
    "tester_results_V2_moreAccurate"
]

for dataset in datasets:
    results_base = f"/Users/petr/Documents/Projects/Python/AlgorithmTester/analysis/{dataset}"
    
    all_results = {
        f'{results_base}/wuf-A/wuf20-88-A': f'{results_base}/wuf-A/wuf20-88-A.dat',
        f'{results_base}/wuf-A/wuf20-91-A': f'{results_base}/wuf-A/wuf20-91-A.dat',

        f'{results_base}/wuf-M/wuf20-78-M': f'{results_base}/wuf-M/wuf20-78-M.dat',
        f'{results_base}/wuf-M/wuf50-201-M': f'{results_base}/wuf-M/wuf50-201-M.dat',

        f'{results_base}/wuf-N/wuf20-78-N': f'{results_base}/wuf-N/wuf20-78-N.dat',
        f'{results_base}/wuf-N/wuf50-201-N': f'{results_base}/wuf-N/wuf50-201-N.dat',

        f'{results_base}/wuf-Q/wuf20-78-Q': f'{results_base}/wuf-Q/wuf20-78-Q.dat',
        f'{results_base}/wuf-Q/wuf50-201-Q': f'{results_base}/wuf-Q/wuf50-201-Q.dat',

        f'{results_base}/wuf-R/wuf20-78-R': f'{results_base}/wuf-R/wuf20-78-R.dat',
        f'{results_base}/wuf-R/wuf50-201-R': f'{results_base}/wuf-R/wuf50-201-R.dat',
    }

    for results_path, output_file_path in all_results.items():
        aggregate_files(results_path, output_file_path)

## Temporary

In [43]:
import os
import pandas

def create_df(path):
    with open(path, "r") as input_file:
        line = input_file.readline()
        df = None
        while line is not None and line != '':
            split = line.split(' ')
            print(f"File: {split[0]}")
            
            split = {i:v for (i, v) in enumerate(split)}
            print(type(split))
            
            if df is None:
                df = pandas.DataFrame([split.values()], columns=split.keys())
            else:
                df2 = pandas.DataFrame([split.values()], columns=split.keys())
                df = df.append(df2)
                
            line = input_file.readline()
            
        return df

path = "/home/petr/Documents/LocalShared/AlgorithmTester/analysis/tester_results_V2_moreAccurate"

#create_df(f'{path}/wuf-R/wuf20-78-R.dat')
pandas.read_csv(f'{path}/wuf-R/wuf20-78-R.dat', index_col=None, delimiter=" ", header=None)\
        .iloc[:, [0, 1, -10,-9,-8,-7,-6,-5,-4,-3,-2,-1]]

,0,1,22,23,24,25,26,27,28,29,30,31
0,uf20-0930,83372,0,20,78,SA_SAT_V1,1000.0,0.5,0.9995,55,835945,32557.117233
1,uf20-0600,13973,0,20,78,SA_SAT_V1,1000.0,0.5,0.9995,55,835945,28975.542895
2,uf20-0784,80463,0,20,78,SA_SAT_V1,1000.0,0.5,0.9995,55,835945,27065.612006
3,uf20-0651,11134,0,20,78,SA_SAT_V1,1000.0,0.5,0.9995,55,835945,27771.107196
4,uf20-0961,153100,0,20,78,SA_SAT_V1,1000.0,0.5,0.9995,55,835945,33718.566816
...,...,...,...,...,...,...,...,...,...,...,...,...
986,uf20-018,35083,0,20,78,SA_SAT_V1,1000.0,0.5,0.9995,55,835945,27467.300826
987,uf20-0919,51925,0,20,78,SA_SAT_V1,1000.0,0.5,0.9995,55,835945,31943.285844
988,uf20-0629,13203,0,20,78,SA_SAT_V1,1000.0,0.5,0.9995,55,835945,32033.812104
989,uf20-0678,8101,0,20,78,SA_SAT_V1,1000.0,0.5,0.9995,55,835945,27633.044430


In [12]:
path = "/home/petr/Documents/LocalShared/AlgorithmTester/analysis/tester_results_V2_moreAccurate"

def get_wrong_lines(path):
    with open(path) as res_file:
        leng = 0
        line = res_file.readline()
        while line is not None and line != '':
            split = line.split()
            
            if leng == 0:
                leng = len(split)
            
            if len(split) != leng:
                print(split[0])
                
            line = res_file.readline()
            
get_wrong_lines(f'{path}/wuf-Q/wuf20-78-Q.dat')

In [ ]:
path = "/home/petr/Documents/LocalShared/AlgorithmTester/data"
parts = ["uf20-088","uf20-033","uf20-0780","uf20-077","uf20-0890","uf20-0341","uf20-0340","uf20-0908","uf20-0909"]


def remove(path):
    print("running")
    for root, _, files in os.walk(path):
        for file in files:
            for part in parts:
                if f'{part}.' in file:
                    os.remove(f'{root}/{file}')
                
remove(f'{path}/wuf-N/wuf20-78-N')

In [7]:
import os
from shutil import move

path = "/home/petr/Documents/LocalShared/AlgorithmTester/analysis/tester_results_V1_lessAccurate"

def remove(path):
    print("running")
    with open(f"{path}_N", "w") as new_file:
        with open(path) as res_file:
            line = res_file.readline()
            while line is not None and line != '':
                split = line.split()
                if len(split) <= 32:
                    new_file.write(line)
                #print(line)
                line = res_file.readline()
    
    print("renaming")
    move(f"{path}_N", path)
                
remove(f'{path}/wuf-N/wuf20-78-N.dat')
remove(f'{path}/wuf-Q/wuf20-78-Q.dat')
remove(f'{path}/wuf-M/wuf20-78-M.dat')
remove(f'{path}/wuf-R/wuf20-78-R.dat')

running
renaming
running
renaming
running
renaming
running
renaming


# Read lines of file in batches

In [ ]:
from itertools import islice

filepath: str = '../data/UnitTest/NK10_inst.dat'

# How many lines are read at once    
batch_size: int = 50
    
with open(filepath, "r") as file:
    batch: List[str] = [x.strip() for x in islice(file, batch_size)]
    while len(batch) > 0:
        for instance in batch:
            print(instance)
        
        batch = [x.strip() for x in islice(file, batch_size)]

In [ ]:
from package_parsers.knapsack_parser import KnapsackParser

def get_batch(parser, file, batch_size):
    batch = list()
    cntr: int = 0
    parsed_data = parser.get_next_instance(file)
    
    while (parsed_data is not None) and cntr < batch_size:
        batch.append(parsed_data)

        cntr += 1
        parsed_data = parser.get_next_instance(file)
        
    return batch

parser: KnapsackParser = KnapsackParser()
filepath: str = '../data/UnitTest/NK10_inst.dat'

# How many lines are read at once    
batch_size: int = 5
    
with open(filepath, "r") as file:
    batch = get_batch(parser, file, batch_size)
    
    while len(batch) > 0:
        print(batch)

# yield test

In [ ]:
def get_instance(cnt: int):
    for i in range(cnt):
        yield i
    
algorithms = ["A", "B", "C"]

for alg in algorithms:
    it = get_instance(5)
    
    for res in it:
        print(f'{alg}:{res}')
        
    print("---------------")

# Dummy ConcurrencyStrategy

In [ ]:
import multiprocessing
import concurrent.futures
import time

# Common functions

def get_res(algorithm: str, instance: Dict[str, object]):
    print(f"Getting res for {instance} using {algorithm}.")
    time.sleep(1)
    return f'Ran {algorithm} for {instance}.'

def get_instance(file: str):
    output = dict()
    
    for i in range(2):
        output = {"input": file, "instance": i}
        yield output
    
def run_tester_for_file(filename: str):
    for algorithm in ["A", "B"]:
        it = get_instance(f'{filename}_opened')
        
        for instance in it:
            res = get_res(algorithm, instance)
            print(f"Writing (\'{res}\') to results file for input file {filename}.")

            
# Runners
filenames = [f'File{i}' for i in range(2)]


class BaseRunner:
    
    def start(self):
        for filename in filenames:
            run_tester_for_file(filename)

class ConcurrentFilesRunner:
    
    def start(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            batch = filenames
            futures = [executor.submit(run_tester_for_file, filename) for filename in batch]
            
class ConcurrentInstancesRunner:
    
    def start(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            
            for filename in filenames:
                for algorithm in ["A", "B"]:
                    batch = [instance for instance in get_instance(f'{filename}_opened')]
                    futures = [executor.submit(get_res, instance, algorithm) for instance in batch]

                    for future in concurrent.futures.as_completed(futures):
                        res = future.result()
                        print(f"Writing (\'{res}\') to results file for input file {filename}.")

In [ ]:
# Test BaseRunner
start = time.perf_counter()

simple_runner = BaseRunner()
simple_runner.start()

finish = time.perf_counter()
print(f'Finished task in {round(finish - start, 2)} second(s)')

In [ ]:
# Test ConcurrentFilesRunner
start = time.perf_counter()

simple_runner = ConcurrentFilesRunner()
simple_runner.start()

finish = time.perf_counter()
print(f'Finished task in {round(finish - start, 2)} second(s)')

In [ ]:
# Test ConcurrentInstancesRunner
start = time.perf_counter()

simple_runner = ConcurrentInstancesRunner()
simple_runner.start()

finish = time.perf_counter()
print(f'Finished task in {round(finish - start, 2)} second(s)')

## Other

In [ ]:
source_path = "/Users/petr/Documents/Projects/Python/AlgorithmTester/analysis/tester_results"
tmp_filepath = "/Users/petr/Documents/Projects/Python/AlgorithmTester/analysis/tester_results/help.tmp"

for root, _, files in os.walk(source_path):
    for file in files:
        if ".mwcnf" not in file:
            continue
        input_filename = f'{root}/{file}'
        
        with open(input_filename, "r") as input_file:
            with open(f'{tmp_filepath}', "w") as tmp_file:
                tmp_file.write(input_file.readline().replace(".cnf", ""))
           
        shutil.move(tmp_filepath, input_filename)